# Exercise 3: Wildcard

You know the drill. Pick *ONE* of the following exercises to complete.

In [ ]:
import torch
import torchvision
from torchvision.datasets import FakeData
from torchvision import transforms
import matplotlib.pyplot as plt
import torch.nn.functional as F
from torch import nn
from torch import optim
import numpy as np
from sklearn import metrics

----
# Exercise 3.1: Implement ODIN for OOD detection
ODIN is a very simple approach, and you can already start experimenting by implementing a temperature hyperparameter in your base model and doing a grid search on $T$ and $\varepsilon$.


In [ ]:
# A very simple CNN model.
class CNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = torch.flatten(x, 1)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

batch_size = 128
dim_fake = 1000
img_size = (3, 32, 32)

ds_train = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
ds_test = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

dl_train = torch.utils.data.DataLoader(ds_train, batch_size=batch_size, shuffle=True)
dl_test = torch.utils.data.DataLoader(ds_test, batch_size=batch_size, shuffle=False)

ds_fake = FakeData(size=dim_fake, image_size=img_size, transform=transform)
dl_fake = torch.utils.data.DataLoader(ds_fake, batch_size=batch_size, shuffle=False)

model = CNN().to(device)

# PRETRAINED MODEL.
model.load_state_dict(torch.load('./cifar10_CNN.pth')) 


In [ ]:
def compute_scores(data_loader, score_fun):
    scores = []
    with torch.no_grad():
        for data in data_loader:
            x, y = data
            output = model(x.to(device))
            s = score_fun(output)
            scores.append(s)
        scores_t = torch.cat(scores)
        return scores_t


def detection(model, data_loader, T, eps, delta):
    model.train()

    score = []
    for (xs, ys) in data_loader:
        xs, ys = xs.to(device), ys.to(device)
        xs.requires_grad=True 
        xs.retain_grad()

        output = model(xs)
        model.zero_grad()
        l = F.cross_entropy(output, ys)
        l.backward()

        # FGSM
        v = xs + eps*torch.sign(xs.grad) 

        with torch.no_grad():
            logit = model(v)
            # Get the max softmax score
            s = F.softmax(logit / T, 1).max(dim=1)[0]

            # Compute the score 
            for i in range(np.shape(s)[0]): 
                if s[i] > delta:
                    score.append(1)
                else:
                    score.append(0)
    score_t = torch.tensor(score)
    return score_t

In [ ]:
# GRID SEARCH
# 10 values for epsilon, 7 values for temperature
e = np.linspace(0, 0.004, 10)
t = np.array([1, 5, 10, 50, 100, 500, 1000])

for eps in e:
    for temp in t:
        scores_fake= detection(model, dl_fake, temp, eps, delta=0.5)
        scores_test= detection(model, dl_test, temp, eps, delta=0.5)
        y_test = torch.ones_like(scores_test)
        y_fake = torch.zeros_like(scores_fake)
        y = torch.cat((y_test, y_fake))
        y_pred = torch.cat((scores_test, scores_fake))
        s = metrics.roc_auc_score(y, y_pred)
        print("AUC for eps = %s, temp = %s is %s" % (eps, temp, s))

----
# Exercise 3.2: Implement JARN
In exercise 2.2 you already implemented Jacobian-regularized learning to make your model more robust to adversarial samples. Add a *discriminator* to your model to encourage the adversarial samples used for regularization to be more *salient*.

See [the JARN paper](https://arxiv.org/abs/1912.10185) for more details.


In [ ]:
# Model
class CNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = torch.flatten(x, 1)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

class Adaptor(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv = nn.Conv2d(3,3,1)
    
    def forward(self, x):
        x = self.conv(x)
        x = F.tanh(x)
        return x
     
class Discriminator(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv_net = nn.Sequential(
            nn.Conv2d(3, 16, 5),
            nn.ReLU(),
            nn.Conv2d(16, 32, 5),
            nn.ReLU(),
            nn.Conv2d(32, 64, 5),
            nn.MaxPool2d(2,2)
            )
        
        self.linear_layers = nn.Sequential(
            nn.Linear(64*10*10, 400),
            nn.ReLU(),
            nn.Linear(400, 10),
            nn.ReLU(),
            nn.Linear(10,1),
            nn.Sigmoid()
        )

    def forward(self, x):
        x = self.conv_net(x)
        x= torch.flatten(x,1)
        x = self.linear_layers(x)
        return x

In [ ]:
# JARN
def training_model(dataloader, epochs, model, apt, disc, lr_model, lr_apt, lr_disc, u):
    model.train()
    apt.train()
    disc.train()

    opt_model = torch.optim.Adam(model.parameters(), lr=lr_model)
    opt_apt = torch.optim.Adam(apt.parameters(), lr=lr_apt)
    opt_disc = torch.optim.Adam(disc.parameters(), lr=lr_disc)

    for epoch in range(epochs):
        tot_loss_avd = 0
        tot_loss = 0
        for (xs, ys) in tqdm(range(dataloader)):
            xs, ys = xs.to(device), ys.to(device)
            xs.requires_grad = True

            output = model(xs)
            xs.retain_grad()

            model.zero_grad()

            loss = nn.CrossEntropyLoss(output, ys)
            loss.backward(retain_graph = True)
            tot_loss += loss.item()

            jac = xs.grad
            xs.requires_grad = False

            generated = apt(jac)

            # Adversarial loss with binary cross entropy
            reals = torch.ones_like(disc(xs))
            fakes = torch.zeros_like(disc(jac))

            loss_reals = nn.BCELoss(reals, disc(xs))
            loss_fake = nn.BCELoss(fakes, disc(generated))
            loss_adv = loss_reals + loss_fake

            tot_loss_avd += loss_adv.item()

            # Training model loss
            l_theta = loss + u*loss_adv   
            l_phi = -1*loss_adv     

            # Update the parameters
            opt_model.zero_grad()
            opt_apt.zero_grad()
            opt_disc.zero_grad()

            l_theta.backward(retain_graph= True)
            loss_adv.backward(retain_graph= True)
            l_phi.backward()

            opt_model.step()
            opt_apt.step()
            opt_disc.step()
            
        print(epoch, tot_loss/len(dataloader),tot_loss_avd/len(dataloader))
    torch.save(model.state_dict(), './model_states/jarn_model.pth')

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

batch_size = 64

ds_train = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
ds_test = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

dl_train = torch.utils.data.DataLoader(ds_train, batch_size=batch_size, shuffle=True)
dl_test = torch.utils.data.DataLoader(ds_test, batch_size=batch_size, shuffle=False)

model = CNN().to(device)
apt = Adaptor().to(device)
disc = Discriminator().to(device)

epochs = 10

lr_model = 0.0001
lr_apt = 0.0001
lr_disc = 0.0001
u=1

training_model(dl_train, epochs, model, apt, disc, lr_model, lr_apt, lr_disc, u)

----
# Exercise 3.3: Experiment with *targeted* adversarial attacks
Implement the targeted Fast Gradient Sign Method to generate adversarial samples that *imitate* samples from a specific class. Evaluate your adversarial samples qualitatively and quantitatively.